# Calculator

- Hold
- No vig probability

In [1]:
from dataclasses import dataclass
import numpy as np

In [2]:
class Market:
    '''
    Sportsbetting convience tools.
    '''
    def __init__(self, line_1=None, line_2=None):
        self.line_1 = line_1
        self.line_2 = line_2
        self.probability_1 = None
        self.probability_2 = None
        self.hold = None
        
        # Calculate the hold
        self.get_hold()
    
    @staticmethod    
    def _get_implied_probability(line):
        '''
        Assuming American Odds
        '''
        if line >= 100:
            return 100 / (100 + line)
        else:
            return line / (-100 + line)
        
    def get_hold(self):
        self.probability_1 = self._get_implied_probability(self.line_1)
        self.probability_2 = self._get_implied_probability(self.line_2)
        self._total_probability = self.probability_1 + self.probability_2
        
        self.hold = 100 * (self._total_probability - 1)
        
        print(f'Hold: {round(self.hold,2)}%')
        
    def get_vig_free_probability(self):
        return self.probability_1 / self._total_probability
    
    @staticmethod
    def convert_american_to_decimal(square_line):
        if square_line > 0:
            return (square_line / 100) + 1
        else:
            return square_line * (100 / square_line ) + 1
        

In [3]:
pinny = Market(line_1=131, line_2=-150)

Hold: 3.29%


In [4]:
square = Market(line_1=165, line_2=-200)

Hold: 4.4%


In [5]:
print('Estimated ROI:')
round(100 * (pinny.get_vig_free_probability() - square.probability_1),1)

Estimated ROI:


4.2

In [6]:
print('Estimated Edge:')
round(100 * (square.convert_american_to_decimal(165) * pinny.get_vig_free_probability() - 1),1)

Estimated Edge:


11.1

## Estimate Push Frequency

- Over the lower number and under the higher number.

In [69]:
six_and_a_half = Market(-122, 108)

Hold: 3.03%


In [70]:
seven_and_a_half = Market(-219, 184)

Hold: 3.86%


In [81]:
# Vig free under on the 7.5 less vig free over on the 6.5

100 * (seven_and_a_half.get_vig_free_probability() - six_and_a_half.get_vig_free_probability())

12.760635257190367

In [75]:
square.get_vig_free_probability()

0.3614457831325302

In [80]:
0.36 * (16.5) + 0.127 * 10  + (1 - square.get_vig_free_probability()) * -10

0.8244578313253008

## Another Market

In [140]:
consensus = Market(115, -135)
consensus.get_vig_free_probability()

Hold: 3.96%


0.447405997144217

In [141]:
square = Market(line_1=146, line_2=-176)
square.get_vig_free_probability()

Hold: 4.42%


0.38930264048747465

In [137]:
print('Estimated Edge:')
round(100 * (square.convert_american_to_decimal(146) * consensus.get_vig_free_probability() - 1),1)

Estimated Edge:


10.1

## Better Class

In [7]:
class Market:
    '''
    Sportsbetting convience tools.
    '''
    def __init__(self, line_1=None, line_2=None):
        self.line_1 = line_1
        self.line_2 = line_2
        self.probability_1 = None
        self.probability_2 = None
        self.hold = None
        self.vig_free_1 = None 
        self.vig_free_2 = None
        
        # Calculate the hold
        self._get_market_stats()
    
    @staticmethod    
    def _get_implied_probability(line):
        '''
        Assuming American Odds
        '''
        if line >= 100:
            return 100 / (100 + line)
        else:
            return line / (-100 + line)
        
    def _get_market_stats(self):
        '''
        Get implied probabilities, the hold, and the vig free probabilities for this market.
        '''
        ## Implied Probabilities:
        self.probability_1 = self._get_implied_probability(self.line_1)
        self.probability_2 = self._get_implied_probability(self.line_2)
        
        ## Hold
        self._total_probability = self.probability_1 + self.probability_2
        self.hold = 100 * (self._total_probability - 1)
        
        ## Vig Gree
        
        self.vig_free_1 = self.probability_1 / self._total_probability
        self.vig_free_2 = self.probability_2 / self._total_probability

    @staticmethod
    def convert_american_to_decimal(line):
        if abs(line) < 100:
            raise ValueError('American Odds must be quoted like -110, +120')
        
        if line > 0:
            return (line / 100) + 1
        else:
            return (-100 / line ) + 1
        
    def calculate_edge(self, square_line):
        return round(100 * (self.convert_american_to_decimal(line=square_line) * self.vig_free_1 - 1),1)
        

In [8]:
pinny = Market(line_1=131, line_2=-150)
print(pinny.hold)
print(pinny.vig_free_1)

3.2900432900432985
0.4191114836546521


In [9]:
square = Market(line_1=165, line_2=-200)
print(square.hold)
print(square.vig_free_1)

4.402515723270439
0.3614457831325302


In [10]:
pinny.calculate_edge(square_line=165)

11.1

In [11]:
consensus_my_side = 165
consensus_other_side = -213
soft_line = 195

consensus = Market(consensus_my_side, consensus_other_side)
consensus.calculate_edge(soft_line)

5.2

In [12]:
consensus_my_side = 165
consensus_other_side = -213
soft_line = 195

consensus = Market(consensus_my_side, consensus_other_side)
consensus.calculate_edge(soft_line)

5.2

In [15]:
consensus_my_side = -110
consensus_other_side = -106
soft_line = 112

consensus = Market(consensus_my_side, consensus_other_side)
consensus.calculate_edge(soft_line)

6.9

In [21]:
consensus_my_side = -205
consensus_other_side = 162
soft_line = -161

consensus = Market(consensus_my_side, consensus_other_side)
consensus.calculate_edge(soft_line)

3.4